# 전이학습(Transfer Learning) 개념 및 필요성
### 학습 데이터 부족
- CNN 모델의 품질을 높이기 위해서는, 즉 임의의 데이터에 대해서 정확도는 높이고 오버피팅은 줄이기 위해서는 기본적으로 많은 양의 데이터를 이용하여 학습해야 함.
- 그러나, 많은 학습 데이터를 확보하려면 많은 비용과 시간이 소요되기 때문에 현실적으로 쉽지 않음. 이러한 데이터가 부족한 어려움을 해결하기 위해 등장한 것이 전이학습(Transfer Learning)임.

### 전이학습(Transfer Learning) 개념 및 필요성
- <b>전이학습(Transfer Learning)</b>이란 아주 큰 dataset,
- 즉 21,841 부류에 대해서 총 1천4백만장 이상의 이미지로 구성된 ImageNet 데이터를 사용해서 학습된 모델의 가중치를 가져와서,
- 우리가 해결하려는 문제에 맞게 보정해서 사용하는 것을 의미함.
- 이때 큰 dataset을 사용해서 훈련된 모델을 <b>사전 학습 모델(pre-trained model)</b>이라고 함.
- ImageNet 데이터의 이미지 크기는 평균적으로 469X387 임.

<img src = "http://cmseng.skku.edu/CMSLecture/ML/img/25-1.png" style="max-width: 60%; height: auto;"><p>

# 사전 학습 모델 (pre-trained model)
<img src = "http://cmseng.skku.edu/CMSLecture/ML/img/25-2.png" style="max-width: 60%; height: auto;"><p>

# Fine-tuning

- 사전 학습 모델의 가중치를 미세하게 조정하는 기법이며, 새롭게 분류하려는 데이터의 종류와 전체 개수를 미리 분석한 후에,
- 그것을 바탕으로 사전 학습 모델 가중치 일부만을 재학습 시키거나 또는 모든 가중치를 처음부터 다시 학습 시킬 수 있음.
- Fine-tuning 진행 시 많은 연산량이 필요하므로 일반적으로 CPU보다는 GPU를 많이 사용함. 
<img src = "http://cmseng.skku.edu/CMSLecture/ML/img/25-3.png" style="max-width: 60%; height: auto;"><p>

# Transfer Learning 사용법

<pre>
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

# tensorflow에서 제공하는 ImageNet으로 사전학습된 다양한 사전학습 모델 VGG16, ResNet50, MobileNet, InceptionV3 등
    
base_model = tf.keras.applications.VGG16(<b>weights='imagenet',</b>  # 사전학습에 사용된 dataset
                                         <b>include_top=False,</b>   # False: 사전학습에 사용된 특징추출기만 가져옴. True: 특징추출기완 분류기 모두 가져옴.
                                         <b>input_shape=(240,240,3))</b> $ 새롭게 학습 시킬 이미지 텐서 크기
base_model.summary() </pre>

#### [[tensorflow에서 제공하는 사전학습 모델]](https://keras.io/api/applications/) https://keras.io/api/applications/
#### [[top-1, top-5 accuracy]](https://enjoyso.tistory.com/122)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

# ImageNet으로 사전학습된 VGG16, ResNet50, MobileNet, InceptionV3 등

base_model = tf.keras.applications.VGG16(weights='imagenet',
                                         include_top=False,
                                         input_shape=(240,240,3))
base_model.summary()

2024-01-12 22:40:05.375495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
58889256/58889256 [==============================] - 7s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 240, 240, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)  

# VGG16 사전학습모델 
<img src = "http://cmseng.skku.edu/CMSLecture/ML/img/25-4.png" style="max-width: 60%; height: auto;"><p>
[[참고자료]](https://89douner.tistory.com/270)

# 새로운 분류기
<pre>
model = Sequential()

model.add(base_model)    # 사전학습 모델의 특징추출기 

model.add(Flatten())     #  Flatten() 함수 이외에 GlobalAveragePooling2D()도 많이 사용됨.

model.add(Dense(64, activation='relu'))        # 새로운 분류기 
model.add(Dropout(0.25))                       #
model.add(Dense(4, activation='softmax'))      # New classifier 

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()    
</pre>

In [2]:
model = Sequential()

model.add(base_model)

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 64)                1605696   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 16,320,644
Trainable params: 16,320,644
Non-trainable params: 0
_________________________________________________________________
